##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# This work has been substantially altered from the original

# Convolutional Neural Networks

Much of the code in this notebook comes from [tensorflow's GitHub](https://github.com/tensorflow/docs/tree/master/site/en/tutorials).

## Goals for Today

- Describe the types of layers that are distinctive for convolutional nets
- Utilize `tensorflow` to build CNNs
- Evaluate CNN models

## What are CNNs?

From [Wikipedia](https://en.wikipedia.org/wiki/Convolutional_neural_network):

- "CNNs are regularized versions of multilayer perceptrons. Multilayer perceptrons usually mean fully connected networks, that is, each neuron in one layer is connected to all neurons in the next layer. The "fully-connectedness" of these networks makes them prone to overfitting data. Typical ways of regularization include adding some form of magnitude measurement of weights to the loss function. However, CNNs take a different approach towards regularization: they take advantage of the hierarchical pattern in data and assemble more complex patterns using smaller and simpler patterns."
<br/>
<br/>
- "Convolutional networks were inspired by biological processes in that the connectivity pattern between neurons resembles the organization of the animal visual cortex. Individual cortical neurons respond to stimuli only in a restricted region of the visual field known as the receptive field. The receptive fields of different neurons partially overlap such that they cover the entire visual field."

### Convolving and Pooling

The two distinctive types of layer inside of a typical CNN are **convolutional** and **pooling** layers. There often several of each of these layers in a network.  Let's look at each in turn.


#### Convolution


Convolutional nets employ [convolutions](https://en.wikipedia.org/wiki/Convolution), which are a certain kind of transformation. In the context of neural networks processing images, this can be thought of as sliding a filter (of weights) over the image matrix to produce a new matrix of values. (I'll detail the calculation below.) The relative smallness of the filter means both that there will be relatively few parameters to learn and that the values representing certain areas of the image will be affected only by the values of *nearby areas*. This helps the network in **feature detection**. Let's check out some visualizations [here](https://www.freecodecamp.org/news/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050/).

![sliding window](https://github.com/learn-co-students/011121-pt-ds/blob/main/phase_4/resources/conv-layer-animation.gif?raw=1)
*image thanks to [Daphne Cornelisse](https://www.freecodecamp.org/news/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050/)


Utkarsh Sinha shows us some examples of different kinds of filters [here](http://aishack.in/tutorials/convolutions/).

#### Hyper parameters:

There are 3 important hyperparameters to consider when designing your convolutional layers:
1. Filter Dimensions
2. Stride
3. Padding

#### 1. Filter Dimensions:

These are simply the shape of your filter.  (2,2) would be a 2 by 2 filter covering 4 total pixels.  (3,3) 3 by 3 and cover 9 pixels.  No matter the size, the filter will return a single value on each step.  The math is shown below.  Also, color images have 3 'channels' or layers, Red, Blue, and Green pixel values.  So, a 3x3 filter for a color image would actually have size 3x3x3.

#### 2. Stride

Stride is how far the filter moves with each step.  A stride of 1 means the filter moves over by 1 pixel on each step, the animation above demonstrates a 3x3 filter with a stride of 1.  A filter with a stride of 2 would move 2 pixels on each step.

#### 3. Padding.  

Padding is technically not a feature of the filter, but is preparation of the data before the filter is passed over it.  A convolutional layer can add a layer of zeros aorund the edge of the image to change the size of the output matrix.  

**Valid padding** means no padding.  The layer will output a smaller matrix than the input.

**Same padding** will output a matrix of the same size as the input matrix.  This is done by adding a layer of zeros all the way around the image of size `filter // 2` or filter size divided by 2, rounded down.  

**Full padding** adds a layer of zeros of size `filter - 1`.  This will output a matrix larger than in input and each pixel will be in each position of the filter kernel.

![padding](https://github.com/learn-co-students/011121-pt-ds/blob/main/phase_4/resources/padding.jpeg?raw=1)


### The Math:


Suppose we have a 3x3 image with no padding and a 2x2 filter and a stride of 1. Then the calculation goes as follows:

$\begin{bmatrix}
a & b & c \\
d & e & g \\
h & j & k
\end{bmatrix} *
\begin{bmatrix}
f_1 & f_2 \\
f_3 & f_4
\end{bmatrix} =
\begin{bmatrix}
f_1a + f_2b + f_3d + f_4e & f_1b + f_2c + f_3e + f_4g \\
f_1d + f_2e + f_3h + f_4j & f_1e + f_2g + f_3j + f_4k
\end{bmatrix}$.

In words: Line up the filter with the image, multiply all the corresponding pairs and then add up those products. Repeat for all positions of the filter as allowed by [the stride and the padding](https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/). The relative position of the filter to the image will tell you which entry in the resultant matrix you're filling in.

Let's try an example of horizontal edge detection. One good filter for that might look like:

$\begin{bmatrix}
10 & 10 & 10 \\
0 & 0 & 0 \\
-10 & -10 & -10
\end{bmatrix}$

Suppose we apply this filter to (i.e. *convolve*) an image with a clear horizontal edge, such as this one:

$\begin{bmatrix}
200 & 200 & 200 & 200 & 200 \\
200 & 200 & 200 & 200 & 200 \\
200 & 200 & 200 & 200 & 200 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0
\end{bmatrix}$

<br/>
<br/>
<details><summary>
    Answer here
    </summary>
    <br/>
    $\begin{bmatrix}
    0 & 0 & 0 \\
    6000 & 6000 & 6000 \\
    6000 & 6000 & 6000 \\
    0 & 0 & 0
    \end{bmatrix}$
    Notice how the edge is now "highlighted"!
    </details>

[Here](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) is another good resource.

#### Pooling
What is pooling? The main goal in inserting a pooling layer is to reduce dimensionality, which helps to reduce both network computation and model overfitting. This is generally a matter of reducing a matrix or tensor of values to  some smaller size, and the most common way of doing this is by partitioning the large matrix into $n$ x $n$ blocks and then replacing each with the largest value in the block. Hence we speak of "MaxPooling".

### Max Pooling in Action

![max pooling example](https://github.com/learn-co-students/011121-pt-ds/blob/main/phase_4/resources/maxpool.png?raw=1)

#### Putting it all together

The combination of convolutional layers and pooling layers compress two dimensions of the image while deepening the last through the repeated application of *multiple filters* per layer.  

Each of the multiple different filters, or kernels, on each layer produce their own output which is generally smaller (in the case of valid padding) in "width" and "height" and the layer stacks the output of each filter to make the 3rd dimension of the matrix, "depth" greater.  

Pooling layers further compress the "height" and "width" by pulling out the most relevant features of the image, those with the greatest activation value.

Densely connected layers are generally added in the last few layers in image classification tasks.

![CNN image classification example](https://github.com/learn-co-students/011121-pt-ds/blob/main/phase_4/resources/cnn_architecture_example.jpeg?raw=1)

## From the TensorFlow Authors

This tutorial demonstrates training a simple [Convolutional Neural Network](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) to classify MNIST digits. This simple network will achieve over 99% accuracy on the MNIST test set. Because this tutorial uses the [Keras Sequential API](https://www.tensorflow.org/guide/keras), creating and training our model will take just a few lines of code.

Note: CNNs train faster with a GPU. If you are running this notebook with Colab, you can enable the free GPU via * Edit -> Notebook settings -> Hardware accelerator -> GPU*.

### Import TensorFlow

In [ ]:
#!pip install tensorflow

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

### Download and prepare the MNIST dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [ ]:
train_images.shape

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(10,5))
axes = axes.ravel()
for i in range(6):
  axes[i].imshow(train_images[i].reshape((28,28)))

plt.show()

In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255, test_images / 255

### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to color channels, MNIST has one (because the images are grayscale), whereas a color image has three (R,G,B). In this example, we will configure our CNN to process inputs of shape (28, 28, 1), which is the format of MNIST images. We do this by passing the argument `input_shape` to our first layer.



In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let display the architecture of our model so far.

In [ ]:
model.summary()

The number of parameters depends on the number of input and output channels of the layer in question. For more, see [this post](https://medium.com/@zhang_yang/number-of-parameters-in-dense-and-convolutional-neural-networks-34b54c2ec349).

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as we go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically,  as the width and height shrink, we can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top
To complete our model, we will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, we will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. MNIST has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

 Here's the complete architecture of our model.

In [ ]:
model.summary()

As you can see, our (3, 3, 64) outputs from the final max pooling layer were flattened into vectors of shape (576) before going through two Dense layers.

### Compile and train the model

In [ ]:
test_labels.shape

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=10, batch_size=1000)

### Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
print(test_acc)

As you can see, our simple CNN has achieved a really high test accuracy. Not bad for a few lines of code!

If you would like to go further with this, including learning how to incrase the size of your image dataset with new, synthetic images using image augmentation, and how to train on more images than you can fit in your RAM, check out [this colab notebook by the team at Google](https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part2.ipynb#scrollTo=SdW6geEVi2S8)

# Conclusion:

In this notebook you learned about convolutional neural networks.  

You learned about their distinctive layers: convolutional and pooling, and how tensor data flows through them and changes shape.

You also learned how to create an CNN in Keras to classify handwritten numerals with high accuracy.